# Quick Start

Загружаем нужные библиотеки.

In [1]:
import requests
import pandas as pd

from opendota import(
    query_opendota,
)

SQL-запрос к базе OpenDota: https://www.opendota.com/explorer

P.S. Если сначала выбрать что-то в полях, а затем нажать Toggle SQL, то автоматически сформируется нужный запрос

Схема БД: https://github.com/odota/core/blob/master/sql/create_tables.sql

In [2]:
ODOTA_QUERY = ''' 
SELECT
    matches.*,
    player_matches.*,
    ((player_matches.player_slot < 128) = matches.radiant_win) win,
    leagues.name leaguename
FROM 
    matches
        JOIN
            match_patch using(match_id)
        JOIN 
            leagues using(leagueid)
        JOIN 
            player_matches using(match_id)
        JOIN 
            heroes on heroes.id = player_matches.hero_id
        LEFT JOIN
            notable_players ON notable_players.account_id = player_matches.account_id AND notable_players.locked_until = (SELECT MAX(locked_until) FROM notable_players)
        LEFT JOIN 
            teams using(team_id)
WHERE TRUE
ORDER BY matches.match_id DESC NULLS LAST
LIMIT 200
'''

Используем функцию query_opendota, которая принимает на вход запрос и возвращает pandas.DataFrame()

In [3]:
df_matches = query_opendota(ODOTA_QUERY)
df_matches.head()

,ability_upgrades_arr,ability_uses,account_id,actions,additional_units,assists,backpack_0,backpack_1,backpack_2,barracks_status_dire,...,times,tower_damage,tower_status_dire,tower_status_radiant,towers_killed,version,win,xp_per_min,xp_reasons,xp_t
0,"[5439, 5440, 5438, 5440, 5440, 5441, 5440, 543...",None,254025588,None,None,6,0,0,0,63,...,None,0,2038,1792,NaN,NaN,False,218,None,None
1,"[5448, 5450, 5450, 5448, 5450, 5452, 5450, 544...",None,129071882,None,None,8,0,0,0,63,...,None,0,2038,1792,NaN,NaN,False,271,None,None
2,"[5401, 5402, 5400, 5400, 5400, 5403, 5400, 540...",None,234699894,None,None,6,46,0,34,63,...,None,353,2038,1792,NaN,NaN,False,421,None,None
3,"[5685, 5090, 5090, 5685, 5090, 5093, 5090, 509...",None,237238721,None,None,7,0,0,46,63,...,None,1742,2038,1792,NaN,NaN,False,449,None,None
4,"[5505, 5505, 5506, 5506, 5505, 5505, 5504, 550...",None,11865327,None,None,3,46,0,0,63,...,None,0,2038,1792,NaN,NaN,False,537,None,None


Смотрим что там в интересного в данных

In [4]:
df_matches.columns

Index(['ability_upgrades_arr', 'ability_uses', 'account_id', 'actions',
       'additional_units', 'assists', 'backpack_0', 'backpack_1', 'backpack_2',
       'barracks_status_dire',
       ...
       'times', 'tower_damage', 'tower_status_dire', 'tower_status_radiant',
       'towers_killed', 'version', 'win', 'xp_per_min', 'xp_reasons', 'xp_t'],
      dtype='object', length=118)

Найти нужного игрока можно по account_id

In [5]:
account_id = 20421681
dotabuff_player = 'https://www.dotabuff.com/players/'
opendota_player = 'https://www.opendota.com/players/'
dotabuff_url = '{}{}'.format(dotabuff_player, account_id)
opendota_url = '{}{}'.format(opendota_player, account_id)
print(dotabuff_url)
print(opendota_url)

https://www.dotabuff.com/players/20421681
https://www.opendota.com/players/20421681


Матч можно искать по match_id

In [6]:
match_id = 3458810313
dotabuff_match = 'https://www.dotabuff.com/matches/'
opendota_match = 'https://www.opendota.com/matches/'
db_url = '{}{}'.format(dotabuff_match, match_id)
odota_url = '{}{}'.format(opendota_match, match_id)
print(db_url)
print(odota_url)

https://www.dotabuff.com/matches/3458810313
https://www.opendota.com/matches/3458810313


Внутри матча каждый игрок сидит на одном из 10 слотов.

У нормальных людей слоты изменяются от 0 до 9,

У opendota слоты в рамках одного матча изменяются от 0 до 4 - The Radiant, а далее от 128 до 132 - The Dire

In [7]:
df_matches[['account_id', 'match_id', 'player_slot']].head(10)

,account_id,match_id,player_slot
0,254025588,3573137571,0
1,129071882,3573137571,1
2,234699894,3573137571,2
3,237238721,3573137571,3
4,11865327,3573137571,4
5,113372833,3573137571,128
6,35504297,3573137571,129
7,91932652,3573137571,130
8,298726580,3573137571,131
9,88826062,3573137571,132


Посчитаем кто больше всех нанес урона по вышкам в одном матче

In [8]:
df_best_tower_damage = (
    df_matches
    .groupby('account_id', as_index=False)
    .max()
    [['tower_damage', 'match_id', 'account_id', 'player_slot']]
    .sort_values('tower_damage', ascending=False)
)
df_best_tower_damage.head()

,tower_damage,match_id,account_id,player_slot
38,9868,3572702903,87586992,130
87,9759,3572702903,123051238,131
19,9365,3572628673,83074695,130
67,9242,3572696340,108376607,132
102,9154,3572814592,153749549,132


Посмотрим глазками на результат

In [9]:
best_match_id = df_best_tower_damage.match_id[0]
db_result_url = '{}{}'.format(dotabuff_match, best_match_id)
result_url = '{}{}'.format(opendota_match, best_match_id)
print(db_result_url)
print(result_url)

https://www.dotabuff.com/matches/3572628673
https://www.opendota.com/matches/3572628673
